In [2]:
import numpy as np
import pandas as pd
from keras.models import load_model
from tensorflow.keras.utils import pad_sequences
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.text import Tokenizer

# Load the model
model = load_model('music_model.h5')

# Load the tokenizer and scaler
tokenizer = Tokenizer()
tokenizer.word_index = np.load('tokenizer_word_index.npy', allow_pickle=True).item()
scaler = StandardScaler()
scaler.mean_, scaler.scale_ = np.load('scaler_mean_scale.npy')

# Define the maximum sequence length
max_sequence_length = 100

# Function to generate merged text
def generate_merged_text(seed_text, num_of_words, diversity, artist, genre, year):
    seed_tokens = tokenizer.texts_to_sequences([seed_text + artist + genre + year])
    seed_tokens = pad_sequences(seed_tokens, maxlen=max_sequence_length)

    numerical_inputs = scaler.transform([[year, num_of_words]])

    generated_text = seed_text

    for _ in range(num_of_words):
        prediction = model.predict([seed_tokens, numerical_inputs])
        prediction = prediction[0][-1]

        prediction /= np.sum(prediction)  # Normalize probabilities

        valid_indices = [idx for word, idx in tokenizer.word_index.items() if idx < len(prediction)]
        output_word_index = np.random.choice(valid_indices, p=prediction[valid_indices] / np.sum(prediction[valid_indices]))
        output_word = tokenizer.index_word.get(output_word_index, '')

        generated_text += ' ' + output_word

        seed_tokens = np.append(seed_tokens[:, 1:], [[output_word_index]], axis=1)

    return generated_text

# Example usage
generated_music = generate_merged_text("I am lonely, I am feeling very sad today", num_of_words=200, diversity=0.5, artist="Ariana Grande", genre="Pop", year="2018")
print(generated_music)


1/1 [==============================] - 0s 31ms/step
I am lonely, I am feeling very sad today hey this would ive right one dream i cat back want its to you dumb do my name hotter you i speak you way be dont dreams zone one lady on i yeah got its make so again less its it i id greatest cant it up lady sky no can so it yourself run dont no know the i up me yeah a again me myself no me in i gon my oh dance just i yeah anybody lie my gonna the broke just her right brand pour i need mary the i know unconditionally liquor love it favorite stones shouldve dont to her i this mmm fly down it i come in how me imagine get got love time woo long wont love ooh love eh killed i get it you swear the bum the this mamamamanicure oh yeah em nigga hold caring how wont dont right dont i im baby just and to me it take dont amo as dont you beyonc i now love way are answer version hey got catfish love why zone i take be la know look on tell name di let good i you check best know my like is meee man           